# Utiliza aprendizado de máquina para predizer evolução para UTI de pacientes de SRAG por influenza

In [2]:
import pandas as pd
import os
import datetime
import dateutil
import numpy as np

pd.set_option('display.max_columns', None)

In [3]:
dados = pd.read_parquet('../influenza_ML_2025-10-30_16-28-12.parquet')